<a href="https://colab.research.google.com/github/TarteTatin1212/contents_search/blob/main/contents_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://chat.openai.com/c/72549bc6-2164-44a1-9da1-8fff0c3cfb9f


In [1]:
# Googleドライブのマウント
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd '/content/drive/MyDrive/Senior_Project'

/content/drive/MyDrive/Senior_Project


In [3]:
!pip install --upgrade google-api-python-client
!pip install pytz
!pip install langid

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 67.3 MB/s eta 0:00:00
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 2.84.0
    Uninstalling google-api-python-client-2.84.0:
      Successfully uninstalled google-api-python-client-2.84.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 33.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langid: filename=langid-1.1.6-py3-none-any.whl size=1941171 sha256=abac7525861b43c6300e1b0a25b18d57f09cba53261c30feb8f20b1e887425d4
  Stored in directory: /root/.cache/pip/wheels/23/c8/c6/eed80894918490a175677414d40bd7c851413bbe03d4856c3c
Successfully bui

In [4]:
from apiclient.discovery import build
from datetime import datetime, timedelta
import pytz
import json
import pandas as pd
import langid

In [5]:
# APIキーを設定
api_key = ""

# APIクライアントを構築
youtube = build("youtube", "v3", developerKey=api_key)

# 日本時間の1時間前をISO 8601形式の文字列に変換
one_hour_ago = (datetime.now(pytz.timezone("Asia/Tokyo")) - timedelta(hours=1)).isoformat()


In [12]:
# 検索リクエストを実行
search_response = youtube.search().list(
    q="",  # 検索クエリ（ここでは全ての動画を対象）
    part="id",  # 動画のIDを取得
    type="video",  # ビデオのみを対象
    publishedAfter=one_hour_ago,  # 1時間前以降に公開された動画のみを対象
    maxResults=100,  # 最大50件の結果を取得
    regionCode="JP",  # 日本で公開された動画のみを対象
).execute()

# データを保存するリスト
data_japanese = []
data_other = []

# 動画ごとに必要な情報を取得
for item in search_response["items"]:
    video_id = item["id"]["videoId"]

    # 動画の詳細情報を取得
    video_response = youtube.videos().list(
        part="snippet,statistics",  # 動画の基本情報と視聴回数を取得
        id=video_id,  # 検索で取得した動画IDを指定
    ).execute()

    for item in video_response["items"]:
        title = item["snippet"]["title"]  # 動画タイトル
        lang, _ = langid.classify(title)
        channel_title = item["snippet"]["channelTitle"]  # チャンネル名
        thumbnail_url = item["snippet"]["thumbnails"]["default"]["url"]  # サムネイルURL
        view_count = item["statistics"]["viewCount"]  # 再生回数

        # 取得した情報を辞書に格納
        video_info = {
            "title": title,
            "channelTitle": channel_title,
            "thumbnailUrl": thumbnail_url,
            "viewCount": view_count
        }

        # タイトルが日本語なら data_japanese に、それ以外なら data_other に追加
        if lang == 'ja':
            data_japanese.append(video_info)
        else:
            data_other.append(video_info)


In [13]:
# JSON形式で保存
with open('youtube_data_japanese.json', 'w') as json_file:
    json.dump(data_japanese, json_file)
with open('youtube_data_other.json', 'w') as json_file:
    json.dump(data_other, json_file)

# CSV形式で保存
df_japanese = pd.DataFrame(data_japanese)
df_japanese.to_csv('youtube_data_japanese.csv', index=False)
df_other = pd.DataFrame(data_other)
df_other.to_csv('youtube_data_other.csv', index=False)